In [8]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import numpy as np
import os.path
ox.config(log_console=True, use_cache=True)
from IPython.display import clear_output

In [9]:
def make_iso_polys(G, center_nodes, edge_buff=50, node_buff=70, infill=False):
    isochrone_polys_district = gpd.GeoDataFrame()
    for n, center_node in center_nodes.iterrows():
        isochrone_polys = {}
        subgraph = nx.ego_graph(G, center_node['node'], undirected=True, radius=center_node['buffer'], distance="length")

        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys['id']=center_node['id']
        isochrone_polys['geom']=(new_iso)
        isochrone_polys['buffer']=center_node['buffer']
        isochrone_polys_district = isochrone_polys_district.append(isochrone_polys, ignore_index=True)
        print('\rВыполнено объектов в районе', len(isochrone_polys_district),'/',len(center_nodes), end='')
    return isochrone_polys_district

In [10]:
if os.path.isfile('isochrone_polys_districts.gpkg'):
    isochrone_polys_districts = gpd.read_file('isochrone_polys_districts.gpkg')   
    done_districts = isochrone_polys_districts.district.drop_duplicates().values
else:
    isochrone_polys_districts = gpd.GeoDataFrame()

In [11]:
done_districts

array(['поселение Филимонковское', 'район Внуково',
       'поселение Внуковское', 'район Новокосино', 'район Замоскворечье',
       'район Митино', 'поселение Сосенское', 'район Силино',
       'район Соколиная Гора', 'район Бирюлёво Западное',
       'район Братеево', 'район Ростокино', 'район Ховрино',
       'район Ивановское', 'район Хамовники', 'Нижегородский район',
       'поселение "Мосрентген"', 'район Выхино-Жулебино',
       'Гагаринский район', 'район Коптево', 'район Восточный',
       'Алтуфьевский район', 'район Тёплый Стан',
       'район Очаково-Матвеевское', 'поселение Новофедоровское',
       'поселение Щаповское', 'район Ново-Переделкино',
       'Тимирязевский район', 'район Чертаново Центральное',
       'поселение Марушкинское', 'район Ясенево', 'поселение Рязановское',
       'район Хорошёво-Мнёвники', 'поселение Вороновское',
       'район Дорогомилово', 'Южнопортовый район',
       'Бескудниковский район', 'Дмитровский район', 'район Царицыно',
       'Войков

In [14]:
objects_raw = gpd.read_file('sport_objects.gpkg')

districts = gpd.read_file('districts.gpkg')
districts = districts[~districts.name.isin(done_districts)]
# districts = districts[districts['name']=='поселение Внуковское']
districts_buffers = districts.copy()


In [7]:
for dn in districts_buffers.name:
    objects = objects_raw.overlay(districts[districts['name']==dn][['name','geometry']], how='intersection')
    polygon_buf = districts_buffers[districts_buffers['name']==dn].to_crs('epsg:32637').buffer(max(objects['buffer'])).to_crs('epsg:4326').geometry.values[0]

    
    # download the street network
    G = ox.graph_from_polygon(polygon_buf, simplify=False, custom_filter=(
        '["highway"]["highway"!~"abandoned|construction|planned|platform|proposed|raceway"]["service"!~"private"]'
    ),)

    nodes = ox.distance.nearest_nodes(G, list(objects['geometry'].x), list(objects['geometry'].y))
    center_nodes = pd.concat([objects['id'], pd.Series(nodes, name='node'), objects['buffer']], axis=1)
    G = ox.project_graph(G)
    clear_output(wait=True)
    print('Выполнено районов',districts[districts['name']==dn].index[0],'/', len(districts))
    len_all_objs = len(isochrone_polys_districts)
    print('Выполнено всего объектов до этого района',len_all_objs,'/', len(objects_raw))

    isochrone_polys_district = make_iso_polys(G, center_nodes, edge_buff=50, node_buff=0, infill=True)
    
    isochrone_polys_district['district'] = dn

    isochrone_polys_district.rename(columns={"geom": "geometry"}, inplace=True)
    isochrone_polys_district.set_geometry('geometry', inplace=True)

    utm_zone = int(np.floor((37.61556 + 180) / 6) + 1)
    utm_crs = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    isochrone_polys_district.set_crs(utm_crs, inplace=True)
    isochrone_polys_district.to_crs(epsg=4326, inplace=True)
    isochrone_polys_district.geometry = isochrone_polys_district.geometry.apply(lambda x:Polygon(x.exterior))
    isochrone_polys_district.geometry = isochrone_polys_district.simplify(0.0001)
    isochrone_polys_districts = isochrone_polys_districts.append(isochrone_polys_district, ignore_index=True)

    isochrone_polys_districts.to_file('isochrone_polys_districts.gpkg', driver='GPKG')



Выполнено районов 143 / 90
Выполнено всего объектов до этого района 10435 / 10521
Выполнено объектов в районе 80 / 80